# Basic imports

In [15]:
import pandas as pd
import os
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams.update({'figure.max_open_warning': 0})

In [16]:
current_path = os.getcwd()
root_path=current_path.replace('\\forward_feature_selection','')

# Loading dataframe

In [17]:
mixed_df=pd.read_csv(root_path+"\\molecules.csv",sep="\t")

# Functions

In [18]:
def save_df_to_disk(df,name:str,separator="\t"):
    df.to_csv(name,sep=separator,index = False, header=True)

In [19]:
def get_data_and_true_prediction(df,not_wanted_features:list):
        temp_df=df.drop(not_wanted_features,axis=1)
        y=temp_df[temp_df.columns[-1]]
        x=temp_df.drop([temp_df.columns[-1]],axis=1)
        
        return x,y   

In [20]:
def get_df_with_name_and_prediction(df,true_prediction,big_df):
    new_df=df
    new_df.insert(0,"m_name",big_df["m_name"].values)
    new_df=new_df.join(true_prediction)
    
    return new_df

# Forward selection

In [21]:
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC

In [22]:
def get_roc_auc_score(x,y,model): # gets roc auc average
        cv_results = cross_validate(model, x, y, cv=10,scoring=('roc_auc'))
        roc_auc_avrg=cv_results['test_score'].mean()
        
        return roc_auc_avrg 

In [23]:
def forward_selection(x,y,model): # O(n) worst case scenario, where n depends on len(x.columns)
    
    def first_iteration(x,y,model):
        score_lst=[]
        for i in range(len(x.columns)):
            k=x.columns[i]
            temp_x=x[[k]]
            score=get_roc_auc_score(temp_x,y,model)
            score_lst.append(score)

        max_score = max(score_lst) # best score
        max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        new_features=[x.columns[i] for i in max_score_index]
        top_new_features=new_features[0]
        best_x=x[top_new_features]
        new_x=x.drop(top_new_features,axis=1)
        
        return best_x,new_x,max_score
    
    def else_iteration(best_x,x,y,model,actual_score):     
        new_x_lenght = len(x.columns)
        if (new_x_lenght > 0):
            score_lst=[]
            for i in range(new_x_lenght):
                k=x.columns[i]
                temp_x=x[[k]]
                temp_new_x=pd.concat([best_x,temp_x],axis=1, ignore_index=True)
                score=get_roc_auc_score(temp_new_x,y,model)
                score_lst.append(score)

            new_max_score = max(score_lst) # best score
            actual_best_score = actual_score # score passed from parameters

            if(new_max_score<actual_best_score):
                return best_x,actual_best_score # break condition, recursive function

            max_score_index=[i for i, j in enumerate(score_lst) if j == new_max_score] # indx with best score 

            new_features=[x.columns[i] for i in max_score_index]
            top_new_features=new_features[0]
            temp_x=x[top_new_features]
            best_x=pd.concat([best_x,temp_x],axis=1)
            new_x=x.drop(top_new_features,axis=1)

            return else_iteration(best_x,new_x,y,model,new_max_score)
        
        return best_x,actual_score
    
    best_x,new_x,score=first_iteration(x,y,model)
    best_x,score=else_iteration(best_x,new_x,y,model,score)
    
    return best_x,score

In [24]:
def forward_partition_SVC(x,y,lst:list,other): # list must be greater than len 2    
    print("\nfirst param list len is {}".format(len(lst)))
    print("first param list -> {}".format(lst))
    
    if (len(lst) == 2):
        first = lst[0]
        print("first is {}".format(first))
        last = lst[1]
        print("last is {}".format(last))
        clf_first = SVC(gamma=other, C=first,random_state=0) 
        first_df,first_score = forward_selection(x,y,clf_first)
        clf_last = SVC(gamma=other, C=last,random_state=0)
        last_df,last_score = forward_selection(x,y,clf_last)
        
        if (last_score > first_score):
            print("last score is greater than first score")
            return last_df,last,last_score
        
        print("first score is greater than last score")
        return first_df,first,first_score
    
    elif(len(lst) == 3):
        first = lst[0]
        print("first is {}".format(first))
        middle = lst[1]
        print("middle is {}".format(middle))
        last = lst[-1]
        print("last is {}".format(last))
        clf_first = SVC(gamma=other, C=first,random_state=0)
        _,first_score = forward_selection(x,y,clf_first)       
        clf_middle = SVC(gamma=other, C=middle,random_state=0)
        _,middle_score = forward_selection(x,y,clf_middle)
        clf_last = SVC(gamma=other, C=last,random_state=0)
        _,last_score = forward_selection(x,y,clf_last)
        
        if ((first_score > middle_score) and (middle_score > last_score)):
            print("sub list taken: left")
            return forward_partition_SVC(x,y,lst[:2],other)  
        
        print("sub list taken: right")            
        return forward_partition_SVC(x,y,lst[1:],other)
    
    else:          
        first = lst[0]
        print("first is {}".format(first))
        middle = lst[(len(lst)//2)-1]
        print("middle is {}".format(middle))
        last = lst[-1]
        print("last is {}".format(last))
        clf_first = SVC(gamma=other, C=first,random_state=0)
        _,first_score = forward_selection(x,y,clf_first)       
        clf_middle = SVC(gamma=other, C=middle,random_state=0)
        _,middle_score = forward_selection(x,y,clf_middle)
        clf_last = SVC(gamma=other, C=last,random_state=0)
        _,last_score = forward_selection(x,y,clf_last)
        
        if ((first_score > middle_score) and (middle_score > last_score)):
            print("sub list taken: left")
            return forward_partition_SVC(x,y,lst[:(len(lst)//2)],other)
        
        print("sub list taken: right")            
        return forward_partition_SVC(x,y,lst[(len(lst)//2)-1:],other)          

In [25]:
def best_SVC(x,y,K:list,P:list):
    
    def get_best_score_index(score_lst:list):
        max_score = max(score_lst) # best score
        max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        
        return max_score_index[0]
    
    best_prm_lst = []
    for i in P:
        print("Seccond parameter is: {}".format(i))
        best_df,best_k,best_score = forward_partition_SVC(x,y,K,i)      
        print("\nBest first parameter is: {}".format(best_k))
        print("ROC AUC = {}".format(best_score))
        print("\nFeatures\n")
        for c,i in enumerate(best_df.columns):
            print("{}. {}".format(c+1,i))
        best_prm_lst.append([best_df,best_k,i,best_score])
        print("")
        
    score_lst = [d  for [a,b,c,d] in best_prm_lst]
    index = get_best_score_index(score_lst)
    print("Final results")
    print("Best model when first param = {} ,second param = {} ,ROC AUC = {}".format(best_prm_lst[index][1],
                                                                 best_prm_lst[index][2],
                                                                 best_prm_lst[index][3]))
    return best_prm_lst[index][0]

In [26]:
def guarantee_results(x,y,K:list,P:list):
    
    def get_best_score_index(score_lst:list):
        max_score = max(score_lst) # best score
        max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        
        return max_score_index[0]
    
    best_outer_prm_lst = []
    for i in K:
        print("First param is {}\n".format(i))
        best_inner_prm_lst = []
        for j in P:
            print("Second param is {}".format(j))
            clf = SVC(random_state=0, C=i,gamma=j) # linear model
            best_df,best_score = forward_selection(x,y,clf)
            print("ROC AUC = {}".format(best_score))
            print("\nFeatures\n")
            for c,d in enumerate(best_df.columns):
                print("{}. {}".format(c+1,d))       
            print("")
            best_inner_prm_lst.append([best_df,i,j,best_score])
            
        score_lst = [d  for [a,b,c,d] in best_inner_prm_lst]
        index = get_best_score_index(score_lst)
        best_outer_prm_lst.append([best_inner_prm_lst[index][0],best_inner_prm_lst[index][1],best_inner_prm_lst[index][2],best_inner_prm_lst[index][3]])
        
    score_lst = [d  for [a,b,c,d] in best_outer_prm_lst]
    index = get_best_score_index(score_lst)
    print("Final results")
    print("Best model when first param = {} ,second param = {} ,ROC AUC = {}".format(best_outer_prm_lst[index][1],
                                                                 best_outer_prm_lst[index][2],
                                                                 best_outer_prm_lst[index][3]))
    return best_outer_prm_lst[index][0]

# Data

In [27]:
unnecesary_features=["m_name"]
x,y = get_data_and_true_prediction(mixed_df,unnecesary_features)

In [13]:
C = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
Gamma = [1e-6,1e-5,1e-4,1e-3,1e-2,1,5,10]

In [14]:
start = time.time()
best_df = best_SVC(x,y,C,Gamma)
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))

Seccond parameter is: 1e-06

first param list len is 20
first param list -> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
first is 1
middle is 10
last is 20
sub list taken: right

first param list len is 11
first param list -> [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
first is 10
middle is 14
last is 20
sub list taken: right

first param list len is 7
first param list -> [14, 15, 16, 17, 18, 19, 20]
first is 14
middle is 16
last is 20
sub list taken: right

first param list len is 5
first param list -> [16, 17, 18, 19, 20]
first is 16
middle is 17
last is 20
sub list taken: right

first param list len is 4
first param list -> [17, 18, 19, 20]
first is 17
middle is 18
last is 20
sub list taken: right

first param list len is 3
first param list -> [18, 19, 20]
first is 18
middle is 19
last is 20
sub list taken: right

first param list len is 2
first param list -> [19, 20]
first is 19
last is 20
first score is greater than last score

Best first parameter is: 

last score is greater than first score

Best first parameter is: 10
ROC AUC = 0.8163537305097204

Features

1. n_HBA
2. n_aliphatic_heterocycles
3. fraction_CSP3
4. n_HOH
5. n_HBD

Seccond parameter is: 10

first param list len is 20
first param list -> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
first is 1
middle is 10
last is 20
sub list taken: left

first param list len is 10
first param list -> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
first is 1
middle is 5
last is 10
sub list taken: left

first param list len is 5
first param list -> [1, 2, 3, 4, 5]
first is 1
middle is 2
last is 5
sub list taken: right

first param list len is 4
first param list -> [2, 3, 4, 5]
first is 2
middle is 3
last is 5
sub list taken: right

first param list len is 3
first param list -> [3, 4, 5]
first is 3
middle is 4
last is 5
sub list taken: right

first param list len is 2
first param list -> [4, 5]
first is 4
last is 5
first score is greater than last score

Best first parameter is

'02:32:06'

In [28]:
k_pivot = 17
p_pivot = 0.001
new_K = [i for i in range(k_pivot-3,k_pivot+4)]
new_P = [0.001,0.01,1]

In [29]:
start = time.time()
g_df = guarantee_results(x,y,new_K,new_P)
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))

First param is 14

Second param is 0.001
ROC AUC = 0.8852435975257255

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_O
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_saturated_rings
9. n_rings
10. n_strict_rotable_bonds
11. n_hetero_atoms
12. n_HBA
13. fraction_CSP3

Second param is 0.01
ROC AUC = 0.9014547769766944

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_HBA
4. m_logp
5. n_atoms_stereo_centers
6. n_O
7. n_hetero_atoms
8. n_primary_carbon_atoms
9. n_hetero_cycles
10. n_HOH
11. n_aliphatic_carbocycles
12. n_saturated_carbocycles
13. n_briged_head_atoms
14. fraction_CSP3

Second param is 1
ROC AUC = 0.7829502249434286

Features

1. n_HBD
2. fraction_CSP3
3. n_non_strict_rotable_bonds
4. n_saturated_carbocycles

First param is 15

Second param is 0.001
ROC AUC = 0.8850955622450158

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_O
4. m_logp
5. n_aromatic_carbocycles
6. n_briged_head_atoms
7. n_amide_bonds
8. n_saturated_rings
9. n_r

'00:26:56'

In [30]:
new_df=get_df_with_name_and_prediction(g_df,y,mixed_df)
save_df_to_disk(new_df,"best_classifier_SVC.csv")

C = 17
Gamma = 0.01
Tol = 0.001
ROC AUC = 0.9022

In [31]:
new_df.head()

,m_name,n_HBD,n_aliphatic_heterocycles,n_HBA,m_logp,n_atoms_stereo_centers,n_O,n_hetero_atoms,n_primary_carbon_atoms,n_HOH,n_hetero_cycles,n_aliphatic_carbocycles,n_saturated_carbocycles,n_briged_head_atoms,fraction_CSP3,is_cns_molecule
0,BUMETANIDE,3,0,5,3.0365,0,7,8,17,4,0,0,0,0,0.235294,0
1,BACLOFEN,2,0,2,1.8570,1,3,4,10,3,0,0,0,0,0.300000,1
2,METYRAPONE,0,0,3,2.6371,0,3,3,14,0,2,0,0,0,0.214286,0
3,METHYLPHENOBARBITAL,1,1,3,1.0426,1,5,5,13,1,1,0,0,0,0.307692,1
4,DULOXETINE,1,0,3,4.6309,1,2,3,18,1,1,0,0,0,0.222222,1
